In [1]:
import pandas as pd
import json

# Download GoEmotions dataset from Hugging Face
from datasets import load_dataset
dataset = load_dataset("go_emotions")

# Convert to DataFrame
df = pd.DataFrame(dataset['train'])




c:\Users\smrit\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
df.head()

,text,labels,id
0,My favourite food is anything I didn't have to...,[27],eebbqej
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj
3,To make her feel threatened,[14],ed7ypvh
4,Dirty Southern Wankers,[3],ed0bdzj


In [24]:
print("Missing values in GoEmotions dataset:")
print(df.isnull().sum())

Missing values in GoEmotions dataset:
text      0
labels    0
id        0
dtype: int64


In [ ]:
label_names = dataset['train'].features['labels'].feature.names
print(label_names)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [11]:
# Keep only relevant columns
df = df[['text', 'labels']]

# Map label indexes to emotion names
emotion_labels = ["admiration", "amusement", "anger", "annoyance", "approval", "caring",
                  "confusion", "curiosity", "desire", "disappointment", "disapproval", 
                  "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief", 
                  "joy", "love", "nervousness", "optimism", "pride", "realization", "relief", 
                  "remorse", "sadness", "surprise", "neutral"]

df["emotions"] = df["labels"].apply(lambda x: [emotion_labels[i] for i in x])
df.head()


,text,labels,emotions
0,My favourite food is anything I didn't have to...,[27],[neutral]
1,"Now if he does off himself, everyone will thin...",[27],[neutral]
2,WHY THE FUCK IS BAYLESS ISOING,[2],[anger]
3,To make her feel threatened,[14],[fear]
4,Dirty Southern Wankers,[3],[annoyance]


In [12]:
# Remove unnecessary columns
df = df[['text', 'emotions']]
print(df.head())
# Save as JSON
df.to_json("goemotions_cleaned.json", orient="records", indent=2)
print("GoEmotions dataset saved!")

                                                text     emotions
0  My favourite food is anything I didn't have to...    [neutral]
1  Now if he does off himself, everyone will thin...    [neutral]
2                     WHY THE FUCK IS BAYLESS ISOING      [anger]
3                        To make her feel threatened       [fear]
4                             Dirty Southern Wankers  [annoyance]
GoEmotions dataset saved!


In [9]:
with open("processed_reviews.json", "r") as f:
    reviews_list = json.load(f)

# Display the first 3 entries
print(json.dumps(reviews_list[:3], indent=2))

[
  {
    "Title": "label2 stuning even for the nongamer",
    "Review": "this sound track was beautiful it paints the senery in your mind so well i would recomend it even to people who hate vid game music i have played the game chrono cross but out of all of the games i have ever played it has the best music it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras it would impress anyone who cares to listen",
    "Label": 1
  },
  {
    "Title": "label2 the best soundtrack ever to anything",
    "Review": "im reading a lot of reviews saying that this is the best game soundtrack and i figured that id write a review to disagree a bit this in my opinino is yasunori mitsudas ultimate masterpiece the music is timeless and im been listening to it for years now and its beauty simply refuses to fadethe price tag on this is pretty staggering i must say but if you are going to buy any cd for this much money this is the only one that i feel would be

In [19]:
import pandas as pd  

# Define file path
review_file = "Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_review.json"

# Read in chunks
chunk_size = 100000  # Adjust based on available RAM
chunks = pd.read_json(review_file, lines=True, chunksize=chunk_size)

# Process first chunk (example)
for chunk in chunks:
    print(chunk.columns)  # View first few rows
    break  # Stop after first chunk to prevent excessive memory usage


Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')


In [1]:
import glob

files = glob.glob("**/yelp_academic_dataset_review.json", recursive=True)
print("Possible file locations:", files)


Possible file locations: ['Yelp-JSON\\Yelp JSON\\yelp_dataset\\yelp_academic_dataset_review.json']


In [2]:
import pandas as pd

# Load the business dataset and keep relevant columns
business_df = pd.read_json("Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json", lines=True)
business_df_cleaned = business_df[['business_id', 'name', 'categories', 'stars']]


# Display the cleaned dataframes
print("Business Data (Cleaned):", business_df_cleaned.head())


Business Data (Cleaned):               business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                                          categories  stars  
0  Doctors, Traditional Chinese Medicine, Naturop...    5.0  
1  Shipping Centers, Local Services, Notaries, Ma...    3.0  
2  Department Stores, Shopping, Fashion, Home & G...    3.5  
3  Restaurants, Food, Bubble Tea, Coffee & Tea, B...    4.0  
4                          Brewpubs, Breweries, Food    4.5  


In [3]:
print("Missing values in yelp dataset:")
print(business_df_cleaned.isnull().sum())

Missing values in yelp dataset:
business_id      0
name             0
categories     103
stars            0
dtype: int64


In [4]:
business_df_cleaned.duplicated().sum()


0

In [13]:
import pandas as pd
import sqlite3

# File paths
business_file = "Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json"
review_file = "Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_review.json"
db_file = "yelp_data.db"

# Connect to SQLite database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Create Business Table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS business (
        business_id TEXT PRIMARY KEY,
        name TEXT,
        categories TEXT,
        stars REAL
    )
""")

# Load business data & insert into database
business_df = pd.read_json(business_file, lines=True, dtype={"business_id": str})
business_df_cleaned = business_df[['business_id', 'name', 'categories', 'stars']]
business_df_cleaned.to_sql("business", conn, if_exists="replace", index=False)

print("Business data saved to SQLite!")

# Create Review Table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS review (
        review_id TEXT PRIMARY KEY,
        business_id TEXT,
        stars REAL,
        text TEXT,
        date TEXT,
        FOREIGN KEY (business_id) REFERENCES business(business_id)
    )
""")

# Close database connection
conn.commit()
conn.close()


Business data saved to SQLite!


In [6]:
import pandas as pd
import json

review_file = "Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_review.json"
valid_reviews = []

# Read JSON line-by-line to detect bad lines
with open(review_file, "r", encoding="utf-8") as f:
    for line in f:
        try:
            review = json.loads(line)  # Attempt to parse JSON
            valid_reviews.append(review)
        except json.JSONDecodeError:
            print("⚠️ Skipping malformed line:", line[:100])  # Print first 100 chars of bad line

# Convert to DataFrame
review_df = pd.DataFrame(valid_reviews)

# Save cleaned data to SQLite
import sqlite3
conn = sqlite3.connect("yelp_data.db")

# Store only relevant columns
cols_needed = ["review_id", "business_id", "stars", "text", "date"]
review_df = review_df[cols_needed]
review_df.to_sql("review", conn, if_exists="replace", index=False)

print("✅ Successfully saved clean reviews to SQLite!")
conn.close()


⚠️ Skipping malformed line: {"review_id":"_LCnev_dX3_X12YhZrvqsw","user_id":"hzEHbzC49hFmFIZxMY47JQ","business_id":"chxWW42Y3lB3
✅ Successfully saved clean reviews to SQLite!


In [15]:
len(review_df)

5567220

In [28]:
import pandas as pd
import json

review_file = "Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_review.json"
valid_reviews = []

# Read JSON line-by-line to detect bad lines
with open(review_file, "r", encoding="utf-8") as f:
    for line in f:
        try:
            review = json.loads(line)  # Attempt to parse JSON
            valid_reviews.append(review)
        except json.JSONDecodeError:
            print("⚠️ Skipping malformed line:", line[:10])  # Print first 10 chars of bad line

# Convert to DataFrame
review_df = pd.DataFrame(valid_reviews)
# Check for missing values in the DataFrame
missing_values = review_df.isnull().sum()

# Print the missing values summary
print("Missing values per column:")
print(missing_values)
# Display columns with missing values
print(missing_values[missing_values > 0])


⚠️ Skipping malformed line: {"review_i
Missing values per column:
review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64
Series([], dtype: int64)


In [33]:
review_df.duplicated().sum()


0

In [27]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("yelp_data.db")

# Fetch two sample records from the business table
business_sample = pd.read_sql("SELECT * FROM business LIMIT 2;", conn)

# Fetch two sample records from the review table
review_sample = pd.read_sql("SELECT * FROM review LIMIT 2;", conn)

# Close connection
conn.close()

# Display results
print(" Business Table Sample:\n", business_sample)
print("\n Review Table Sample:\n", review_sample)


 Business Table Sample:
               business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   

                                          categories  stars  
0  Doctors, Traditional Chinese Medicine, Naturop...    5.0  
1  Shipping Centers, Local Services, Notaries, Ma...    3.0  

 Review Table Sample:
                 review_id             business_id  stars  \
0  KU_O5udG6zpxOg-VcAEodg  XQfwVwDr-v0ZS3_CbbE5Xw    3.0   
1  BiTunyQ73aT9WBnpR9DZGw  7ATYjTIgM3jUlt4UM3IypQ    5.0   

                                                text                 date  
0  If you decide to eat here, just be aware it is...  2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year...  2012-01-03 15:28:18  


In [7]:
from datasets import load_dataset
dataset1 = load_dataset("imdb")
df1 = pd.DataFrame(dataset1['train'])
df1.head()

c:\Users\smrit\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [31]:
print("Missing values in imdb dataset:")
print(df1.isnull().sum())

Missing values in imdb dataset:
text     0
label    0
dtype: int64


In [32]:
df1.duplicated().sum()


96

In [2]:
import pandas as pd
import os
import json

def save_json(data, filename):
    with open(f"data/{filename}.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

# Load Amazon Reviews Dataset from local file
amazon_reviews_file = "D:/Customer Emotion Analysis/Customer-Emotion-Analysis/amazon/train.ft.txt/train.ft.txt"

# Read the dataset
with open(amazon_reviews_file, "r", encoding="utf-8") as f:
    amazon_reviews = f.readlines()

# Display basic information about the dataset
print(f"Total Reviews: {len(amazon_reviews)}")
print("\nFirst 5 Rows:")
print(amazon_reviews[:5])


Total Reviews: 3600000

First 5 Rows:
['__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^\n', "__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every pen

In [8]:
import sqlite3
import re

conn = sqlite3.connect("yelp_data.db")
cursor = conn.cursor()

In [9]:
# 1️ Create Table for Amazon Reviews (If not exists)
cursor.execute("""
CREATE TABLE IF NOT EXISTS amazon_reviews (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    review TEXT NOT NULL,
    label INTEGER NOT NULL
);
""")

def process_amazon_data(file_path):
    records = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            # Extract label (either __label__1 or __label__2)
            match = re.match(r"__(label__\d)\s(.+)", line.strip())
            if match:
                label_text, full_review = match.groups()
                
                # Convert FastText label to numeric (1 for negative, 2 for positive)
                label = 1 if label_text == "__label__1" else 2
                
                # Split title and review text
                if ": " in full_review:
                    title, review = full_review.split(": ", 1)
                else:
                    title, review = "No Title", full_review
                
                records.append((title, review, label))
    
    return records

In [10]:
# 3️ Load Amazon Reviews into SQLite
amazon_reviews = process_amazon_data("D:/Customer Emotion Analysis/Customer-Emotion-Analysis/amazon/train.ft.txt/train.ft.txt")
cursor.executemany("INSERT INTO amazon_reviews (title, review, label) VALUES (?, ?, ?)", amazon_reviews)
conn.commit()
print(f" Successfully stored {len(amazon_reviews)} Amazon reviews in yelp_data.db")
conn.close()

 Successfully stored 3600000 Amazon reviews in yelp_data.db


In [11]:
import sqlite3
import pandas as pd
from datasets import load_dataset

# Load IMDB dataset
dataset1 = load_dataset("imdb")
df1 = pd.DataFrame(dataset1['train'])  # Convert Hugging Face dataset to Pandas DataFrame

# Connect to SQLite database
conn = sqlite3.connect("yelp_data.db")
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS imdb_reviews")

cursor.execute("""
CREATE TABLE imdb_reviews (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    text TEXT NOT NULL,
    label INTEGER NOT NULL
);
""")

df1.to_sql("imdb_reviews", conn, if_exists="replace", index=False)

conn.close()
print(" IMDB data successfully replaced in yelp_data.db")


 IMDB data successfully replaced in yelp_data.db


In [4]:
import sqlite3

# Connect to the SQLite database
db_path = "yelp_data.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get the list of all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Available Tables in yelp_data.db:\n")
for table in tables:
    table_name = table[0]
    print(f"🔹 Checking Table: {table_name}")

    # Query first 2 records from each table
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 2;")
    records = cursor.fetchall()

    # Fetch column names
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = [col[1] for col in cursor.fetchall()]

    # Print results
    print("\n Sample Records:\n")
    for row in records:
        for col_name, col_value in zip(columns, row):
            print(f"{col_name}: {col_value}")
        print("-" * 50)
    
    print("\n" + "=" * 60 + "\n")

# Close database connection
conn.close()


Available Tables in yelp_data.db:

🔹 Checking Table: business

 Sample Records:

business_id: Pns2l4eNsfO8kk83dixA6A
name: Abby Rappoport, LAC, CMQ
categories: Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists
stars: 5.0
--------------------------------------------------
business_id: mpf3x-BjTdTEA3yCZrAYPw
name: The UPS Store
categories: Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services
stars: 3.0
--------------------------------------------------


🔹 Checking Table: review

 Sample Records:

review_id: KU_O5udG6zpxOg-VcAEodg
business_id: XQfwVwDr-v0ZS3_CbbE5Xw
stars: 3.0
text: If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. 

The food is good, but it takes a very long time to come out. The waitstaff is very young, but u